In [1]:
import pandas as pd
import glob
import numpy as np
from sklearn import linear_model

In [2]:
frames = [pd.read_csv(file, index_col=0) for file in glob.glob('Data/*.csv')]
df = pd.concat(frames)
df.head()

,V_Current,U_Current,SBG EKF Velocity N,SBG EKF Velocity E,SBG EKF Velocity N Acc,SBG EKF Velocity E Acc,SBG EKF Latitude,SBG EKF Longitude,SBG Heading2,OD19_TRUE_WIND_SPEED,OD19_TRUE_WIND_DIRECTION,ADCP_DC3_V2,ADCP_DC4_V2
Time,,,,,,,,,,,,,
1.474111e+09,-0.019539,-0.312082,16.272120,0.220167,0.098420,0.098593,32.696253,-117.807653,357.98,7.338659,291.86526,16.389969,16.409409
1.474111e+09,-0.019536,-0.312305,16.415127,0.158491,0.098165,0.098338,32.696329,-117.807652,357.94,7.077789,294.14322,16.432737,16.393857
1.474112e+09,-0.019534,-0.312528,16.536410,-0.020466,0.097941,0.098115,32.696405,-117.807651,357.86,7.108053,297.96850,16.459953,16.477449
1.474112e+09,-0.019532,-0.312755,16.517061,-0.082083,0.098028,0.098201,32.696482,-117.807652,357.82,7.539567,300.75003,16.551322,16.339425
1.474112e+09,-0.019530,-0.312981,16.462260,-0.039651,0.098249,0.098422,32.696558,-117.807652,357.79,8.194389,301.61215,16.605754,16.362753


We can do this either in the ship's reference frame or global reference frame as long as we do the transformations. It's the authors opinion that its easier to work in the reference frame of the ship.

In [3]:
df.index = np.round(df.index - df.index[0])
cos_hdg = np.cos(np.deg2rad(df['SBG Heading2']))
sin_hdg = np.sin(np.deg2rad(df['SBG Heading2']))
v_east = df['SBG EKF Velocity E']
v_north = df['SBG EKF Velocity N']
v_lat = v_east*cos_hdg - v_north*sin_hdg
v_fwd = v_east*sin_hdg + v_north*cos_hdg
df['cos_hdg'] = cos_hdg
df['sin_hdg'] = sin_hdg
df['v_lat'] = v_lat
df['v_fwd'] = v_fwd
print(np.mean(v_fwd), np.mean(v_lat))

16.33204590047138 0.3759393880839431


We'll split up the dataframe into two parts that we'll concatenate later. We'll also add in categorical constants and our minimization targets

In [4]:
df_fwd = df.copy()
df_lat = df.copy()
ones = np.ones(shape=v_lat.shape)
zeros = np.zeros(shape=v_lat.shape)
df_fwd['v_stw_fwd'] = ones
df_fwd['v_stw_lat'] = zeros
df_lat['v_stw_fwd'] = zeros
df_lat['v_stw_lat'] = ones #if it can be assuemd the ship has no side slip switch this to zero, or verify with cross-validation
df_fwd['y'] = v_fwd
df_lat['y'] = v_lat

And we'll add the columns needed for a constant current

In [5]:
df_fwd['x_current'] = -1*cos_hdg
df_fwd['y_current'] = sin_hdg
df_lat['x_current'] = sin_hdg
df_lat['y_current'] = cos_hdg

If you assume the current is linearly changing, uncomment the following lines, this can be verified with cross-validation as well

In [6]:
#df_fwd['x_current_c1'] = -1*cos_hdg*df.index
#df_fwd['y_current_c1'] = sin_hdg*df.index
#df_lat['x_current_c1'] = sin_hdg*df.index
#df_lat['y_current_c1'] = cos_hdg*df.index

We'll reconcatenate our dataframes and create our X and y

In [7]:
df_stw = pd.concat([df_fwd, df_lat]) 
y = df_stw.loc[:, ['y']]
columns = ['v_stw_fwd', 'v_stw_lat', 'x_current', 'y_current']
X = df_stw.loc[:, columns]

In [8]:
reg = linear_model.LinearRegression(fit_intercept=False)
reg.fit(X, y)
for coef, col in zip(reg.coef_.tolist()[0], columns):
    print(f"{col}: {coef:0.2f} kts")

v_stw_fwd: 16.31 kts
v_stw_lat: 0.39 kts
x_current: -0.11 kts
y_current: -0.06 kts


Let's calculate the uncertainty using a block bootstrap

In [9]:
bootstrap_count = 1000
samples = np.empty(shape=(len(columns), bootstrap_count))
for i in range(bootstrap_count):
    n = len(df)
    block_size = 30
    prob_block = 1/block_size
    random_sample = np.random.rand(n)
    new_idx = np.random.randint(0, n, n)
    for k in range(1,n):
        if random_sample[k]>prob_block:
            new_idx[k] = new_idx[k-1] + 1
            if new_idx[k] == n:
                new_idx[k] = 0
    new_idx = np.hstack([new_idx, new_idx+n-1])
    new_X = X.iloc[new_idx, :]
    new_y = y.iloc[new_idx, :]
    reg = linear_model.LinearRegression(fit_intercept=False)
    reg.fit(new_X, new_y)
    samples[:, i] = reg.coef_

In [10]:
from scipy import stats

for coef, col in zip(samples, columns):
    coef_mean, coef_std = stats.norm.fit(coef)
    print(f"{col}: {coef_mean:0.2f} +/- {1.96*coef_std:0.2f} kts")

v_stw_fwd: 16.31 +/- 0.02 kts
v_stw_lat: 0.39 +/- 0.03 kts
x_current: -0.11 +/- 0.03 kts
y_current: -0.06 +/- 0.03 kts


In [19]:
adcp_speed = (df['ADCP_DC3_V2']+df['ADCP_DC4_V2'])/2
adcp_speed = adcp_speed[abs(adcp_speed - np.mean(adcp_speed))< 2 * np.std(adcp_speed)]
stats.norm.fit(adcp_speed)

(16.371700358819602, 0.12787046018499879)